In [1]:
# import statements
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

# import MechDesign Helpers
import rm_python_lib.MechDesign.Helpers as HM

from rm_python_lib.MechDesign.Units.Units import m_, mm_, kg_, s_, N_, rpm_, W_, deg_
import rm_python_lib.MechDesign.Units.UnitMethods as UM

import rm_python_lib.MechDesign.RnM as RnM

### 1. Calculate Nominal Torque using Team Specific Data and Motor Size Selected 


$$P_{M,required,SF} = 5500.0W$$

$$K_A = 1.638$$



In [7]:
# new shaft connection object
SC = RnM.ShaftConnection() 

# constants
motor_speed = 1500*rpm_
P_motor = 5500*W_
SC.K_A = 1.638

# convert motor speed
n_M = (1500 / 60)

# calculate Nominal Torque
SC.T_nom = P_motor / (2 * np.pi * n_M)
# SC.T_eq = SC.T_nom * SC.K_A
HM.EqPrint("T_nom",SC.T_nom * N_*m_ / W_)
SC.T_eq = SC.E12_1_hC_DynamicLoadTorque()
HM.EqPrint("T_eq",SC.T_eq * N_*m_ / W_)


Eq(T_nom, 35.01*N_*m_)

Eq(T_eq, 57.35*N_*m_)

Eq(T_eq, 57.35*N_*m_)

### 2. Calculate pgem
Our selected motor size = 132S is 38mm (from Table 16-21)

Standardized Key for motor shaft 38mm from Table 12-2a:
- Width b = 10mm
- Height h = 8mm

Using TB1-1, C45E  
ReNShaft = 295 N/mm^2
ReNKey = 490 N/mm^2
ReN = 295 MPa

$$R_e = K_t * R_{eN}$$

In [ ]:
# constants based on Motor Selection
SC.d = 38*mm_ 
SC.phi = 1 # 1key
SC.n = 1 # 1 key
SC.K_lambda = 1 # method C

SC.K_t = 0.92
SC.b = 10*mm_
SC.h = 8*mm_
Re = 



